In [ ]:
'''
Collaborative filtering: Choose top N books to recommend to a user


Process:
    1. Find K nearest neighbors of a user
    2. Fill in unrated products by taking a weighted average of nearest neighbors that have rated the product (more similar = more weight)
    3. Sort unrated products by their estimated ratings in descending order
    4. Pick top 10

Workflow:
    1. Set up the data
    2. Construct a rating matrix
    3. Find K nearest neighbors
    4. Find top N recommendations

Distance metrics
    - Euclidean distance
    - Correlation distance = 1 - correlation
    - Hamming distance (how many numbers match) = % of numbers in disagreement


Data source: http://www2.informatik.uni-freiburg.de/~cziegler/BX/
'''

'''
Latent Factor Analysis

    Split UserProduct matrix into 2 tables, one for users, the other for products
        - UserProduct = [User] * [Product]
        - [User] = Users x Factors describing user
        - [Product] = Products x Factors describing product
        - In UserProduct matrix, rating is equal to matrix product of User*Product for that user and product
    
    Standard optimization used in Gradient Descent
        R = P*Q
        Error(u,i) = R(u,i) - (P(u) * Q(i))
        Total Square Error = SUM((R(u,i) - (P(u) * Q(i)))^2) # sum of squared error
        MIN(total square error) # want to minimize square error
        Regularization Term = lambda * (||P(u)||^2 + ||Q(i)||^2) # used to prevent overfitting
    
        => MIN(SUM(square error + regularization term))

'''

In [9]:
# Get data (same process as Collaborative_Filtering_KNN.ipynb)
import pandas as pd

dataFile = 'data/BX-Book-Ratings.csv'
data = pd.read_csv(dataFile, sep=';', header=0, names=["user","isbn","rating"], encoding='ANSI')

bookFile = 'data/BX-Books.csv'
books = pd.read_csv(bookFile, sep=';', header=0, names=["isbn","title","author"], encoding='ANSI', error_bad_lines=False, usecols=[0,1,2], index_col=0)

# Get rid of any inconsistent ISBN's since data is not perfect
original_size = data.shape
data = data[data["isbn"].isin(books.index)]
new_size = data.shape

print('Original data shape: {}'.format(original_size))
print('Data shape after removing inconsistencies: {}'.format(new_size))

# Get number of users per ISBN
usersPerISBN = data.isbn.value_counts()
# ISBNs per user
ISBNsPerUser = data.user.value_counts()

# Reduce sparsity of data matrix by:
#   - excluding any isbns that have less than 10 users/ratings
#   - excluding any users that have less than 10 books read
data = data[data["isbn"].isin(usersPerISBN[usersPerISBN > 10].index)]
data = data[data["user"].isin(ISBNsPerUser[ISBNsPerUser > 10].index)]
print(data.shape)

Original data shape: (1149780, 3)
Data shape after removing inconsistencies: (1031175, 3)
(405709, 3)


In [10]:
from scipy.sparse import coo_matrix

# Create internal map of UserIDs to integer codes
data['user'] = data['user'].astype("category")
data['isbn'] = data['isbn'].astype("category")

# coo_matrix((values, (row_source, column_source)))
R = coo_matrix((data['rating'].astype(float),
                    (data['user'].cat.codes.copy(),
                     data['isbn'].cat.codes.copy())))

# Can read from coo_matrix like so: 
print('R.data[0] = {}'.format(R.data[0]))
print('R.row[0] = {}'.format(R.row[0]))
print('R.col[0] = {}'.format(R.col[0]))

R.data[0] = 0.0
R.row[0] = 10633
R.col[0] = 3053


In [3]:
# M = number of users, N = number of ISBNs
M,N = R.shape
# K = number of factors
K = 3

import numpy as np
P = np.random.rand(M,K) # Initialize values for P and Q
Q = np.random.rand(K,N)

from numpy.linalg import norm
def error(R,P,Q,lamda=0.02):
    ratings = R.data
    rows = R.row
    cols = R.col
    e = 0
    for ui in range(len(ratings)):
        rui = ratings[ui]
        u = rows[ui]
        i = cols[ui]
        if rui > 0:
            e = e + pow(rui - np.dot(P[u,:],Q[:,i]), 2) +\
                lamda * (pow(norm(P[u,:]), 2) + pow(norm(Q[:,i]), 2))
    
    return e

# Calculate error
error(R,P,Q)

19513914.866341911

In [4]:
rmse = np.sqrt(error(R,P,Q) / len(R.data))
rmse

4.3501678589274633

In [11]:
def stochaisticGradientDescent(R, K, lamda=0.02, steps=10, gamma=0.001, threshold=0.5):
    '''
    Uses Stochaistic Gradient descent to calculate P,Q matrices and minimize RMSE
    
        1. Set max # of iterations to perform gradient descent
        2. Initial P,Q matrices using random values
        3. Begin gradient descent loop
            - Calculate error between R[ui] and P[u]*Q[i]
            - Update P by small value: P = P + gamma * (partial derivative of error function in terms of P)
            - Update Q by small value: Q = Q + gamma * (partial derivative of error function in terms of Q)
            - Notes:
                - gamma = learning constant
                - lambda = regularization factor
                - error = (R(u,i)-(P(u)*Q(i)))^2 + lambda*(||P(u)||^2+||Q(i)||^2)
        4. If completed max number of iterations or RMSE is below threshold, end gradient descent
        5. Return P,Q
        
    param R: table with (partially) filled data
    param K: number of factors
    param lamda: regularization factor
    param steps: max number of iterations to perform gradient descent
    param gamma: learning rate
    param threshold: satisfactory rmse in which gradient descent can be stopped
    '''
    
    M,N = R.shape
    P = np.random.rand(M,K) # Initialize values for P and Q
    Q = np.random.rand(K,N)
    
    rmse = np.sqrt(error(R,P,Q) / len(R.data))
    print('Initial RMSE: {}'.format(rmse))
    
    for step in range(steps): 
        print('Executing step {}...'.format(step))
        for ui in range(len(R.data)):
            rui = R.data[ui]
            u = R.row[ui]
            i = R.col[ui]
            
            if rui > 0:
                eui = rui - np.dot(P[u,:],Q[:,i])
                P[u,:] = P[u,:] + gamma * 2 * (eui * Q[:,i] - lamda * P[u,:]) # Partial derivative of error function in terms of P
                Q[:,i] = Q[:,i] + gamma * 2 * (eui * P[u,:] - lamda * Q[:,i]) # Partial derivative of error function in terms of Q
        
        rmse = np.sqrt(error(R,P,Q) / len(R.data))
        if rmse < threshold:
            break
            
    print('Final RMSE: {}'.format(rmse))
    return P,Q

In [12]:
stochaisticGradientDescent(R,K=2,gamma=0.0007,lamda=0.01,steps=100)

Initial RMSE: 4.327172212249701
Executing step 0...
Executing step 1...
Executing step 2...
Executing step 3...
Executing step 4...
Executing step 5...
Executing step 6...
Executing step 7...
Executing step 8...
Executing step 9...
Executing step 10...
Executing step 11...
Executing step 12...
Executing step 13...
Executing step 14...
Executing step 15...
Executing step 16...
Executing step 17...
Executing step 18...
Executing step 19...
Executing step 20...
Executing step 21...
Executing step 22...
Executing step 23...
Executing step 24...
Executing step 25...
Executing step 26...
Executing step 27...
Executing step 28...
Executing step 29...
Executing step 30...
Executing step 31...
Executing step 32...
Executing step 33...
Executing step 34...
Executing step 35...
Executing step 36...
Executing step 37...
Executing step 38...
Executing step 39...
Executing step 40...
Executing step 41...
Executing step 42...
Executing step 43...
Executing step 44...
Executing step 45...
Executing st

(array([[ 0.72587866,  1.38930468],
        [ 2.04571476,  1.87924749],
        [ 2.18914234,  2.5119368 ],
        ..., 
        [ 0.9062367 ,  0.59521126],
        [ 2.3074146 ,  2.10651759],
        [ 1.14989965,  1.96354295]]),
 array([[ 1.60545525,  2.21989211,  1.96204847, ...,  2.56073085,
          2.62108258,  0.48051239],
        [ 2.06939785,  1.94346057,  1.77563545, ...,  2.01334575,
          1.71514793,  0.55561289]]))